In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Load the MNIST dataset (for the classification task)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the images to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape to add the channel dimension (grayscale images)
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Randomly generated regression targets (e.g., some numeric value prediction)
# For simplicity, let's create random regression targets
train_regression_targets = np.random.rand(train_images.shape[0], 1)
test_regression_targets = np.random.rand(test_images.shape[0], 1)

# Model architecture
input_layer = layers.Input(shape=(28, 28, 1))  # MNIST input shape

# Shared feature extractor
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)

# Task 1: Classification (10 digits)
classification_output = layers.Dense(10, activation='softmax', name='classification')(x)

# Task 2: Regression (e.g., predict a random value)
regression_output = layers.Dense(1, name='regression')(x)

# Create the MTL model
model = models.Model(inputs=input_layer, outputs=[classification_output, regression_output])

# Compile the model with different losses for each task
model.compile(optimizer='adam',
              loss={'classification': 'sparse_categorical_crossentropy', 'regression': 'mse'},
              metrics={'classification': 'accuracy', 'regression': 'mse'})

# Print the model summary
model.summary()

# Train the model
history = model.fit(train_images,
                    {'classification': train_labels, 'regression': train_regression_targets},
                    epochs=5, batch_size=64,
                    validation_data=(test_images, {'classification': test_labels, 'regression': test_regression_targets}))
# Evaluate the model
loss, classification_loss, regression_loss = model.evaluate(test_images, {'classification': test_labels, 'regression': test_regression_targets})

# Access metrics from the history object
classification_acc = history.history['classification_accuracy'][-1]
regression_mse = history.history['regression_mse'][-1]

print(f"Classification Accuracy: {classification_acc}")
print(f"Regression MSE: {regression_mse}")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 28, 28, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 26, 26, 32)     │            320 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 13, 13, 32)     │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 11, 11, 64)     │         18,496 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 5, 5, 64)       │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 1600)           │              0 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │        204,928 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification (Dense)    │ (None, 10)             │          1,290 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ regression (Dense)        │ (None, 1)              │            129 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 225,163 (879.54 KB)

 Trainable params: 225,163 (879.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 54s 56ms/step - classification_accuracy: 0.8566 - loss: 0.6375 - regression_mse: 0.1162 - val_classification_accuracy: 0.9839 - val_loss: 0.1464 - val_regression_mse: 0.0927
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 52s 56ms/step - classification_accuracy: 0.9843 - loss: 0.1488 - regression_mse: 0.0947 - val_classification_accuracy: 0.9857 - val_loss: 0.1325 - val_regression_mse: 0.0896
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - classification_accuracy: 0.9895 - loss: 0.1254 - regression_mse: 0.0893 - val_classification_accuracy: 0.9911 - val_loss: 0.1158 - val_regression_mse: 0.0866
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 52s 55ms/step - classification_accuracy: 0.9918 - loss: 0.1147 - regression_mse: 0.0882 - val_classification_accuracy: 0.9910 - val_loss: 0.1152 - val_regression_mse: 0.0883
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 53s 56ms/step - classification_accuracy: 0.9937 - loss: 0.1052 - regression_mse: 0.0855 - val_classification_